<a href="https://colab.research.google.com/github/lhajabi/PPI_Grad/blob/main/Dataset_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""ONLY RUNS IF GPU AVAILABLE """
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

gpu()

Found GPU at: /device:GPU:0


<tf.Tensor: shape=(), dtype=float32, numpy=1350.4606>

Imports:

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.2 MB/s eta 0:00:00


In [ ]:
import requests
import gzip
import csv
import io
import sys
import re
import random
import pandas as pd
import tensorflow as tf
import numpy as np
from Bio import SeqIO
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Positive dataset construction

upload data:

In [ ]:
df_Two_Proteins_score = pd.read_csv('/content/drive/MyDrive/Data/9606.protein.links.v12.0.txt', delimiter='\t')
df_Two_Proteins_score[['protein1', 'protein2', 'combined_score']] = df_Two_Proteins_score['protein1 protein2 combined_score'].str.split(expand=True)
df_Two_Proteins_score.drop(columns=['protein1 protein2 combined_score'], inplace=True)

fasta_file_path = '/content/drive/MyDrive/Data/9606.protein.sequences.v12.0.fa'
sequences = []
for record in SeqIO.parse(fasta_file_path, 'fasta'):
    sequences.append({'Header': record.id, 'Sequence': str(record.seq)})
df_sequences = pd.DataFrame(sequences)


CD HIT:

In [ ]:
cd = pd.read_csv('/content/drive/MyDrive/Data/40_tr.txt', delimiter=',', header=None)
cd.rename(columns={0: 'sequence'}, inplace=True)
cd

In [ ]:
#positive_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY POSITIVE/ALL_POS(0.4).txt', delimiter=',')
#positive_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY POSITIVE/ALL_POS(0.7).txt', delimiter=',')
#positive_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD40(0.4).txt', delimiter=',')
#positive_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD40(0.7).txt', delimiter=',')
#positive_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY POSITIVE/ALL_POS_C060(0.4).txt', delimiter=',')
#positive_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY POSITIVE/ALL_POS_CD60(0.7).txt', delimiter=',')

functions:

In [ ]:
def merge_and_transform_data(df_Two_Proteins_score, df_sequences):
    merged_df = df_Two_Proteins_score.merge(df_sequences, left_on='protein1', right_on='Header', how='left')
    merged_df = merged_df.merge(df_sequences, left_on='protein2', right_on='Header', how='left', suffixes=('_protein1', '_protein2'))
    merged_df.drop(columns=['Header_protein1', 'Header_protein2', 'protein1', 'protein2'], inplace=True)
    merged_df.rename(columns={'Sequence_protein1': 'protein1', 'Sequence_protein2': 'protein2'}, inplace=True)
    desired_order = ['protein1', 'protein2', 'combined_score']
    merged_df = merged_df[desired_order]
    return merged_df

def _scale_down_dataframe(input_df, combined_score, total_samples=1000, random_seed=42):
    if combined_score == 1:
        group_ones = input_df[input_df['combined_score'] == 1]
        sample_ones = group_ones.sample(n=min(total_samples, len(group_ones)), random_state=random_seed)
        return sample_ones
    else:
        group_zeros = input_df[input_df['combined_score'] == 0]
        sample_zeros = group_zeros.sample(n=min(total_samples, len(group_zeros)), random_state=random_seed)
        return sample_zeros

def _filter_df(input_df, cd):
    to_concat = []
    for index, row in input_df.iterrows():
        protein1 = row['protein1']
        protein2 = row['protein2']
        mask_protein1 = cd['sequence'] == protein1
        mask_protein2 = cd['sequence'] == protein2
        if mask_protein1.any() and mask_protein2.any():
            to_concat.append(pd.DataFrame({'protein1': [protein1], 'protein2': [protein2]}))
    if to_concat:
        filtered_df = pd.concat(to_concat, ignore_index=True)
    else:
        filtered_df = pd.DataFrame(columns=['protein1', 'protein2'])
    return filtered_df

Code:

In [ ]:
all_positive_df = merge_and_transform_data(df_Two_Proteins_score, df_sequences)
all_positive_df['combined_score'] = pd.to_numeric(all_positive_df['combined_score'], errors='coerce')
all_positive_df['combined_score'] = all_positive_df['combined_score'].div(1000, fill_value=0)
all_positive_df

,protein1,protein2,combined_score
0,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MDLMNGQASSVNIAATASEKSSSSESLSDKGSELKKSFDAVVFDVL...,0.173
1,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MHVMNCVSLVSDKENGNIATAPGFMIGQTPPPAPPPPPPPPPPSPP...,0.154
2,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MALSDADVQKQIKHMMAFIEQEANEKAEEIDAKAEEEFNIEKGRLV...,0.151
3,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MEDGVYEPPDLTPEERMELENIRRRKQELLVEIQRLREELSEAMSE...,0.471
4,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...,MEGRSAAAETFVWVNNASAHSQSVAKAKYEFLFGRSEGKAPDTSDH...,0.201
...,...,...,...
13715399,MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQ...,MEEFDLVKTLHKTSSSVGSDENSLHSLGLNLNTDRSSPHLSTNGVS...,0.195
13715400,MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQ...,MSPKPRASGPPAKATEAGKRKSSSQPSPSDPKKKTTKVAKKGKAVR...,0.158
13715401,MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQ...,MVKMTRSKTFQAYLPSCHRTYSCIHCRAHLANHDELISKSFQGSQG...,0.226
13715402,MAEEQQQPPPQQPDAHQQLPPSAPNSGVALPALVPGLPGTEASALQ...,MRSSKLQSPSPSQEKQGVYLQETAMETWSVEQVCSWLVEKNLGELV...,0.169


In [ ]:
import os
drive_path = '/content/drive/MyDrive/Data'

if not os.path.exists(drive_path):
    os.makedirs(drive_path)

os.chdir(drive_path)
all_positive_df.to_csv('TEMP_ALLPOS_fractions.txt', index=False) #save your df to txt file

In [ ]:
#all_positive_df['combined_score'] = np.where(all_positive_df['combined_score'] >= 0.400, 1, 0)### 0.4 or 0.7
print(all_positive_df['combined_score'].value_counts())

In [ ]:
positive_df = all_positive_df[all_positive_df['combined_score'] == 1]
positive_df

In [ ]:
positive_df = _filter_df(all_positive_df, cd)
positive_df

# Negative dataset construction

Functions

In [ ]:
#returns a csv_reader object from an api
#or if an error occurs prints the error message and exits
def gzip_tsv_api(url):
  response = requests.get(url)

  if response.status_code == 200:
      gzip_content = io.BytesIO(response.content)

      with gzip.GzipFile(fileobj=gzip_content, mode='rb') as gzipped_file:
          tsv_content = gzipped_file.read().decode('utf-8')
          reader = csv.reader(tsv_content.splitlines(), delimiter='\t')
          return reader
  else:
      print(f"Failed to retrieve data. Status code: {response.status_code}")
      sys.exit(1)

In [ ]:
#Get the columns in a csv file #print the columns to know the entries in each column
def _csv_columns(reader):
  columns = [list(column) for column in zip(*reader)]
  return columns

In [ ]:
#returns a csv_reader object from file name
def _tsv_file(file):
 return pd.read_csv(file, sep='\t')


In [ ]:
#print the first items in a dictionary
def _preview_dict(dict, number=5):
  count = 0
  for key, value in seq_loc.items():
      print(f'{key}: {value}')
      count += 1
      if count == number:
          break

In [ ]:
#these functions just format the subcellular location
#i think there's a way to avoid all this
#will look into it if time permits
def _format_subcellular_location_0(location):
  location = re.sub(r'{[^}]*}', '', location)
  note_index = location.find("Note")
    # If "Note" is found, remove everything from it onwards
  if note_index != -1:
      location = location[:note_index]
  return location
def _format_subcellular_location_1(subcellular_location):
  for i in range(len(subcellular_location)):
    subcellular_location[i] = _format_subcellular_location_0(subcellular_location[i])
    subcellular_location[i] = re.sub(r'SUBCELLULAR LOCATION: ', '', subcellular_location[i])
    subcellular_location[i]= subcellular_location[i].split('.')
    subcellular_location[i]= subcellular_location[i][:-1]

def _format_subcellular_location(subcellular_location):
  _format_subcellular_location_1(subcellular_location)
  for i in range(len(subcellular_location)):
    for y in range(len(subcellular_location[i])):
      subcellular_location[i][y] = subcellular_location[i][y].strip()

In [ ]:
# Create a dictionary where seq_loc[sequence] = [subcellular_location]
def create_seq_loc(sequence,subcellular_location):
  seq_loc= {}
  for i in range(len(sequence)):
      key = sequence[i]
      [value] = [subcellular_location[i]]
      seq_loc[key] = value
  return seq_loc

In [ ]:
#each sequence can have a max number of interactions it takes part in #num_seq_interactions is a dictionary where (key = sequence , value = #interactions its in)
def _check_interaction_limit(seq1,seq2,max_num_interactions,num_seq_interactions):
  try:
    if num_seq_interactions[seq1] >= max_num_interactions:
      return 0
  except:
    num_seq_interactions[seq1] = 0
  try:
    if num_seq_interactions[seq2] >= max_num_interactions:
      return 0
  except:
    num_seq_interactions[seq2] = 0
  num_seq_interactions[seq1]+=1
  num_seq_interactions[seq2]+=1
  return 1

In [ ]:
#count the frequency of each subcellular location #output is {subcellular location1: it's frequency percentage, ..., subcellular location N: it's frequency percentage}
def create_loc_freq(seq_loc):
    loc_freq = {}
    total_locations = 0
    for key, values in seq_loc.items():
        for value in values:
            loc_freq[value] = loc_freq.get(value, 0) + 1
    for values in loc_freq.values():
      total_locations += values
    for key, value in loc_freq.items():
      loc_freq[key] = value/total_locations
    return loc_freq

In [ ]:
#recieves two sequences and checks if they have similar subcellular location #function returns 1 if the locations are different
def _check_loc_diff(seq1,seq2,seq_loc):
  for location1 in seq_loc[seq1]:
    for location2 in seq_loc[seq2]:
      if location1 == location2:
        return 0
  return 1

In [ ]:
#this function removes duplicate tuples from a list of tuples, regardless of order #has high memory usage
def _remove_duplicate_tuples_1(lst):
  seen = set()
  result = []
  for pair in lst:
      # Convert the tuple to a frozenset, which is hashable
      frozen_pair = frozenset(pair)
      if frozen_pair not in seen:
          result.append(pair)
          seen.add(frozen_pair)
  return result

In [ ]:
#this function removes duplicate tuples from a list of tuples, regardless of order #has high time usage because of sort function
def  _remove_duplicate_tuples_2(lst):
    seen = set()
    result = []

    for pair in lst:
        # Sort the tuple so that order doesn't matter
        sorted_pair = tuple(sorted(pair))
        if sorted_pair not in seen:
            result.append(pair)
            seen.add(sorted_pair)

    return result

In [ ]:
#This function returns random samples created according to the percentages of each subcellular_location
#for example locations that have a higher percentage will be in more interactions
# samples = [(seq1, location1), ... ]
def create_samples(seq_loc, loc_freq, num_samples, loop_count=3):
  instances = []
  weights = []
  for seq, locations in seq_loc.items():
    for location in locations:
      instances.append((seq, location))
      weights.append(loc_freq[location])

  samples = random.choices(instances, weights = weights, k = num_samples)
  samples = list(set(samples))
  #can add something like this, make sure loop isnt infinite
  for i in range(loop_count):
    if len(samples) < num_samples: #if number of unique instances is less than required num_samples
      new_samples = random.choices(instances, weights = weights, k = num_samples - len(samples))
      new_samples = list(set(new_samples))
      samples.extend(new_samples)
  return samples

In [ ]:
#returns a list of unique negative sample tuples
#each tuple is (sequence1, sequence2)
#where sequence1 and sequence2 have different subcellular locations
#negative_samples = [(sequence1, sequence2), ...]
def create_negative_samples(samples, max_num_interactions, seq_loc):
  num_seq_interactions = {} #num_seq_interactions[seq] = number of interactions
  negative_samples = []
  for i in range(len(samples)):
    for j in range(i + 1, len(samples)):
      if _check_interaction_limit(samples[i][0],samples[j][0], max_num_interactions,num_seq_interactions):
        #samples[i][0] is seq1
        #samples[i][1] is loc1
        if _check_loc_diff(samples[i][0], samples[j][0], seq_loc):
          negative_samples.append((samples[i][0], samples[j][0]))
  negative_samples = _remove_duplicate_tuples_1(negative_samples)
  return negative_samples

In [ ]:
#inputs are:
#a link to a txt file of specific sequences
#seq_loc dictionary containing both representative and non representative sequences
#function will update seq_loc dictionary to contain sequences contained in tr_file only
def tr_seq_loc(link, seq_loc):
  response = requests.get(link)
  if response.status_code == 200:
      file_content = response.text
  else:
      print("Failed to retrieve the file. Status code:", response.status_code)
  file_content = file_content.splitlines()
  tr = []
  for line in file_content:
      tr.append(line)
  tr_loc ={}
  for seq, loc in seq_loc.items():
    if seq in tr:
      tr_loc[seq] = loc
  seq_loc = tr_loc
  return seq_loc

Code

In [ ]:
url = 'https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Clength%2Csequence%2Ccc_subcellular_location%2Ccc_similarity&format=tsv&query=accession%3AP0DOY5+OR+accession%3AP0DPI4+OR+accession%3AP0DPR3+OR+accession%3AQ712N7+OR+accession%3AQ8IVG9&sort=length+asc'
url = 'https://rest.uniprot.org/uniprotkb/stream?compressed=true&fields=accession%2Clength%2Csequence%2Ccc_subcellular_location&format=tsv&query=%28human%29+AND+%28model_organism%3A9606%29+AND+%28proteins_with%3A50%29&sort=length+asc'
reader = gzip_tsv_api(url)
#Define any needed columns from file or url here
#Check print(columns) to know column order in reader
columns = _csv_columns(reader)
length = columns[1][1:] #length not needed?
sequence = columns[2][1:]
#add a function to chanage sequence list by removing similar sequences
subcellular_location = columns[3][1:]
_format_subcellular_location(subcellular_location)
seq_loc = create_seq_loc(sequence,subcellular_location)

In [ ]:
print("length before: ", len(seq_loc))
link = 'https://raw.githubusercontent.com/PPI-Proj/PPI_files/main/40_tr.txt'
seq_loc = tr_seq_loc(link, seq_loc) #representative sequenes only CD
print("length after: " , len(seq_loc))

In [ ]:
num_samples = 500000 #number of samples that random.choice will make
max_num_interactions = 10 #maximum number of interactions per protein sequence, in future turn into percentage
loop_count =  5 #after removing duplicates from samples, recall random.choice
loc_freq = create_loc_freq(seq_loc)
samples = create_samples(seq_loc, loc_freq, num_samples, loop_count)
negative_samples = create_negative_samples(samples, max_num_interactions, seq_loc)
negative_df = pd.DataFrame(negative_samples, columns=['protein1', 'protein2'])
negative_df['combined_score'] = 0
#negative_df.info()

In [ ]:
#negative_df = pd.read_csv('/content/drive/MyDrive/Data/ONLY NEGATIVE/ALL_NEG_CD.txt', delimiter=',')
negative_df.info()

In [ ]:
#Drop duplicates from negative_df if exist in string
all_positive_df['sorted_pair'] = all_positive_df[['protein1', 'protein2']].apply(lambda x: tuple(sorted(x)), axis=1)
negative_df['sorted_pair'] = negative_df[['protein1', 'protein2']].apply(lambda x: tuple(sorted(x)), axis=1)
common_rows = pd.merge(all_positive_df['sorted_pair'], negative_df['sorted_pair'], how='inner')
negative_df = negative_df[~negative_df['sorted_pair'].isin(common_rows['sorted_pair'])]

all_positive_df = all_positive_df.drop(columns='sorted_pair')
negative_df = negative_df.drop(columns='sorted_pair')

negative_df.info()

# Merged Dataset:

In [ ]:
final_df = pd.concat([positive_df, negative_df])

final_df = final_df.reset_index(drop=True)
print(final_df['combined_score'].value_counts())
final_df.info()

Save Final Dataset:

In [ ]:
import os
drive_path = '/content/drive/MyDrive/Data'
# Create the directory if it doesn't exist
if not os.path.exists(drive_path):
    os.makedirs(drive_path)

# Change the current working directory to your Google Drive directory
os.chdir(drive_path)
#final_df.to_csv('sampleName.txt', index=False)
final_df